In [1]:
import os

import cv2
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.nn import CTCLoss

from dataset import Synth90kDataset, synth90k_collate_fn
from model import CRNN
from evaluate import evaluate
from config import train_config as config

### 训练辅助函数

### 注意

1. 尺度

- images: (N, C, H, W) -> (32, 1, 32, 100)
- targets：(X, ),一维向量，长度不固定。32个batch标签（向量）拼接成一个长向量，target_lengths指定切割位置
- logits：(T, N, n_class) -> (24, 32, 37)
- input_lengths: (N,) -> (32, ), 向量中每一个值表识相应batch输入序列长度,为T即24,表示输入序列长度固定为24
- target_lengths: (N, ) ->(32, ), 向量中每一个值表示相应batch训练图像label长度，长度不固定

2. CTCLoss函数使用说明。

1) 获取CTCLoss()对象

```
ctc_loss = nn.CTCLoss(blank=len(CHARS)-1, reduction='mean')
```

- blank：空白标签所在的label值，默认为0，需要根据实际的标签定义进行设定；

- reduction：处理output losses的方式，string类型，可选'none' 、 'mean' 及 'sum'，'none'表示对output losses不做任何处理，'mean' 则对output losses取平均值处理，'sum'则是对output losses求和处理，默认为'mean' 。

2) 在迭代中调用CTCLoss()对象计算损失值
```
loss = ctc_loss(log_probs, targets, input_lengths, target_lengths)
```

- log_probs：shape为(T, N, C)的模型输出张量，其中，T表示CTCLoss的输入长度也即输出序列长度，N表示训练的batch size长度，C则表示包含有空白标签的所有要预测的字符集总长度，log_probs一般需要经过torch.nn.functional.log_softmax处理后再送入到CTCLoss中；

- targets：shape为(N, S) 或(sum(target_lengths))的张量，其中第一种类型，N表示训练的batch size长度，S则为标签长度，第二种类型，则为所有标签长度之和，但是需要注意的是targets不能包含有空白标签；

- input_lengths：shape为(N)的张量或元组，但每一个元素的长度必须等于T即输出序列长度，一般来说模型输出序列固定后则该张量或元组的元素值均相同；

- target_lengths：shape为(N)的张量或元组，其每一个元素指示每个训练输入序列的标签长度，但标签长度是可以变化的；

In [2]:
def train_batch(crnn, data, optimizer, criterion, device):
    crnn.train()
    images, targets, target_lengths = [d.to(device) for d in data]

    logits = crnn(images)
    log_probs = torch.nn.functional.log_softmax(logits, dim=2)

    batch_size = images.size(0)
    input_lengths = torch.LongTensor([logits.size(0)] * batch_size)
    target_lengths = torch.flatten(target_lengths)

    loss = criterion(log_probs, targets, input_lengths, target_lengths)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

### 超参数设置

In [3]:
epochs = config['epochs']
train_batch_size = config['train_batch_size']
eval_batch_size = config['eval_batch_size']
lr = config['lr']
show_interval = config['show_interval']
valid_interval = config['valid_interval']
save_interval = config['save_interval']
cpu_workers = config['cpu_workers']
reload_checkpoint = config['reload_checkpoint']
valid_max_iter = config['valid_max_iter']

img_width = config['img_width']
img_height = config['img_height']
data_dir = config['data_dir']

num_class = len(Synth90kDataset.LABEL2CHAR) + 1

### 判断cuda是否可用，是则基于GPU训练，否则用cpu训练

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

device: cpu


### 设置训练数据加载器、验证数据加载器，常规操作

#### collate_fn用法
- 把list类型batch（若不加此操作，原先一个batch中的多个sample是在list中的）转换成Tensor，加速运算。
- 经collate_fn处理，返回符合需求的数据及相应尺度。

In [5]:
train_dataset = Synth90kDataset(root_dir=data_dir, mode='train',
                                    img_height=img_height, img_width=img_width)
valid_dataset = Synth90kDataset(root_dir=data_dir, mode='dev',
                                img_height=img_height, img_width=img_width)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=cpu_workers,
    collate_fn=synth90k_collate_fn)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=eval_batch_size,
    shuffle=True,
    num_workers=cpu_workers,
    collate_fn=synth90k_collate_fn)

### 实例化CRNN模型，加载模型参数，并运行至可用设备（CPU or GPU）

In [6]:
crnn = CRNN(1, img_height, img_width, num_class,
            map_to_seq_hidden=config['map_to_seq_hidden'],
            rnn_hidden=config['rnn_hidden'],
            leaky_relu=config['leaky_relu'])
if reload_checkpoint:
    crnn.load_state_dict(torch.load(reload_checkpoint, map_location=device))
crnn.to(device)

CRNN(
  (cnn): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu0): ReLU(inplace=True)
    (pooling0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace=True)
    (pooling1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU(inplace=True)
    (pooling2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu4): ReLU(inplace=True)
    (conv5): Conv2d(512, 512, 

### 定义优化方法、损失函数
CTC 的全称是Connectionist Temporal Classification，中文名称是“连接时序分类”，这个方法主要是解决神经网络label 和output 不对齐的问题（Alignment problem），其优点是不用强制对齐标签且标签可变长，仅需输入序列和监督标签序列即可进行训练，目前，该方法主要应用于场景文本识别（scene text recognition）、语音识别（speech recognition）及手写字识别（handwriting recognition）等工程场景。以往我们在百度上搜索pytorch + ctc loss得到的结果基本上warp-ctc的使用方法，warp-ctc是百度开源的一个可以应用在CPU和GPU上高效并行的CTC代码库，但是为了在pytorch上使用warp-ctc我们不仅需要编译其源代码还需要进行安装配置，使用起来着实麻烦。而在Pytorch 1.0.x版本内早就有内置ctc loss接口了，我们完全可以直接使用

In [7]:
optimizer = optim.RMSprop(crnn.parameters(), lr=lr)
criterion = CTCLoss(reduction='sum')
criterion.to(device)

CTCLoss()

In [8]:
assert save_interval % valid_interval == 0
i = 1

### 开始训练

In [9]:
for epoch in range(1, epochs + 1):
    print(f'epoch: {epoch}')
    tot_train_loss = 0.
    tot_train_count = 0
    for train_data in train_loader:
        loss = train_batch(crnn, train_data, optimizer, criterion, device)
        train_size = train_data[0].size(0)

        tot_train_loss += loss
        tot_train_count += train_size
        if i % show_interval == 0:
            print('train_batch_loss[', i, ']: ', loss / train_size)

        if i % valid_interval == 0:
            evaluation = evaluate(crnn, valid_loader, criterion,
                                  decode_method=config['decode_method'],
                                  beam_size=config['beam_size'])
            print('valid_evaluation: loss={loss}, acc={acc}'.format(**evaluation))

        if i % save_interval == 0:
            prefix = 'crnn'
            loss = evaluation['loss']
            save_model_path = os.path.join(config['checkpoints_dir'],
                                           f'{prefix}_{i:06}_loss{loss}.pt')
            torch.save(crnn.state_dict(), save_model_path)
            print('save model at ', save_model_path)

        i += 1

    print('train_loss: ', tot_train_loss / tot_train_count)

epoch: 1
train_batch_loss[ 10 ]:  26.11614227294922
train_batch_loss[ 20 ]:  25.46373748779297
train_batch_loss[ 30 ]:  26.30978012084961
train_batch_loss[ 40 ]:  25.977890014648438
train_batch_loss[ 50 ]:  25.009597778320312


KeyboardInterrupt: 